# Base Notebook

Input data files are available in the read-only "../input/" directory
For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Imports

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os

## Look at the data

In [2]:
# Print all files in the input directory

neutrino_folder = 'E:\\Neutrino'

for dirname, _, filenames in os.walk(neutrino_folder) :
    for filename in filenames:
        print(os.path.join(dirname, filename))

E:\Neutrino\sample_submission.parquet
E:\Neutrino\sensor_geometry.csv
E:\Neutrino\test_meta.parquet
E:\Neutrino\train_meta.parquet
E:\Neutrino\test\batch_661.parquet
E:\Neutrino\train\batch_1.parquet
E:\Neutrino\train\batch_10.parquet
E:\Neutrino\train\batch_100.parquet
E:\Neutrino\train\batch_101.parquet
E:\Neutrino\train\batch_102.parquet
E:\Neutrino\train\batch_103.parquet
E:\Neutrino\train\batch_104.parquet
E:\Neutrino\train\batch_105.parquet
E:\Neutrino\train\batch_106.parquet
E:\Neutrino\train\batch_107.parquet
E:\Neutrino\train\batch_108.parquet
E:\Neutrino\train\batch_109.parquet
E:\Neutrino\train\batch_11.parquet
E:\Neutrino\train\batch_110.parquet
E:\Neutrino\train\batch_111.parquet
E:\Neutrino\train\batch_112.parquet
E:\Neutrino\train\batch_113.parquet
E:\Neutrino\train\batch_114.parquet
E:\Neutrino\train\batch_115.parquet
E:\Neutrino\train\batch_116.parquet
E:\Neutrino\train\batch_117.parquet
E:\Neutrino\train\batch_118.parquet
E:\Neutrino\train\batch_119.parquet
E:\Neutrin

In [3]:
# Look at the first parquet file

train_batch_example = pd.read_parquet(neutrino_folder + '\\train\\batch_1.parquet')
train_batch_example.head()

,sensor_id,time,charge,auxiliary
event_id,,,,
24,3918,5928,1.325,True
24,4157,6115,1.175,True
24,3520,6492,0.925,True
24,5041,6665,0.225,True
24,2948,8054,1.575,True


In [4]:
# look at the sample submission file
sample_submission = pd.read_parquet(neutrino_folder + '\\sample_submission.parquet')
sample_submission.head()

,event_id,azimuth,zenith
0,2092,1,1
1,7344,1,1
2,9482,1,1


In [5]:
# look at the test file
test = pd.read_parquet(neutrino_folder + '\\test\\batch_661.parquet')
test.head()

,sensor_id,time,charge,auxiliary
event_id,,,,
2092,4066,6170,1.275,True
2092,3512,6374,0.975,True
2092,897,6378,1.475,True
2092,2060,6590,0.925,True
2092,3072,6625,1.075,True


In [6]:
# look at the train metadata file
train_metadata = pd.read_parquet(neutrino_folder + '\\train_meta.parquet')
train_metadata.head()

,batch_id,event_id,first_pulse_index,last_pulse_index,azimuth,zenith
0,1,24,0,60,5.029555,2.087498
1,1,41,61,111,0.417742,1.549686
2,1,59,112,147,1.160466,2.401942
3,1,67,148,289,5.845952,0.759054
4,1,72,290,351,0.653719,0.939117


In [7]:
# look at the test metadata file
test_metadata = pd.read_parquet(neutrino_folder + '\\test_meta.parquet')
test_metadata.head()

,batch_id,event_id,first_pulse_index,last_pulse_index
0,661,2092,0,298
1,661,7344,299,334
2,661,9482,335,377


In [8]:
# look at the sensor geometry csv file
sensor_geometry = pd.read_csv(neutrino_folder + '\\sensor_geometry.csv')
sensor_geometry.head()

,sensor_id,x,y,z
0,0,-256.14,-521.08,496.03
1,1,-256.14,-521.08,479.01
2,2,-256.14,-521.08,461.99
3,3,-256.14,-521.08,444.97
4,4,-256.14,-521.08,427.95
